In [ ]:
'''
    转移学习,采用部分的模型,再加上自定义的输入层和辨识层,就能够不受限于模型原先辨识的对象,也就是转移学习(transfer learning)

    使用ResNet152V2模型,辨识花朵数据集
    加载ResNet部分模型 ----> 加载flower数据集  ---> 特征缩放 ---> 建立模型 ---> 模型训练  --->预测
'''
import tensorflow as tf
from keras.applications.resnet_v2 import  ResNet152V2
from keras.preprocessing import image
from keras.applications.resnet_v2 import preprocess_input
from keras.applications.resnet_v2 import decode_predictions
from keras.layers import Dense,GlobalAveragePooling2D
from keras.models import Model
import numpy as np

mnist = tf.keras.datasets